In [1]:
import os
import sys
import multiprocessing
import logging
import numpy as np
import pandas as pd

import mxnet as mx
from mxnet.io import DataDesc
from mxnet import nd, gluon, autograd
from mxnet.gluon.data import RecordFileDataset, ArrayDataset, Dataset
from mxnet.gluon.data.vision import transforms
from mxnet.gluon.data.vision.datasets import ImageFolderDataset
from mxnet.gluon.data.dataloader import DataLoader
from mxnet.gluon.model_zoo import vision as models
from mxnet import recordio

from sklearn.metrics.ranking import roc_auc_score
from sklearn.model_selection import train_test_split
from PIL import Image
from common.utils import *
from common.params_dense import *
import math
from time import time

%load_ext autoreload
%autoreload 2

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("MXNet: ", mx.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
Numpy:  1.13.3
MXNet:  1.2.0
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 9.1.85
CuDNN Version  7.1.3


In [3]:
# User-set
# Note if NUM_GPUS > 1 then MULTI_GPU = True and ALL GPUs will be used
# Set below to affect batch-size
# E.g. 1 GPU = 64, 2 GPUs =64*2, 4 GPUs = 64*4
# Note that the effective learning-rate will be decreased this way
CPU_COUNT = multiprocessing.cpu_count()
GPU_COUNT = len(get_gpu_name())
MULTI_GPU = GPU_COUNT > 1
print("CPUs: ", CPU_COUNT)
print("GPUs: ", GPU_COUNT)

CPUs:  32
GPUs:  4


In [4]:
# Manually scale to multi-gpu
if MULTI_GPU:
    LR *= GPU_COUNT
    BATCHSIZE *= (GPU_COUNT)
    BATCHSIZE = BATCHSIZE//GPU_COUNT*GPU_COUNT

## Data Download

In [5]:
# Model-params
# Paths
CSV_DEST = "/data/chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")

In [6]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 468 ms, sys: 136 ms, total: 604 ms
Wall time: 604 ms


## Data prep
https://github.com/apache/incubator-mxnet/issues/1480


In [7]:
df = pd.read_csv(LABEL_FILE)
df.head()    

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [8]:
def data_prep(df, img_dir, patient_ids):
    # Split labels on unfiltered data
    df_label = df['Finding Labels'].str.split(
        '|', expand=False).str.join(sep='*').str.get_dummies(sep='*')

    # Filter by patient-ids (both)
    df_label['Patient ID'] = df['Patient ID']
    df_label = df_label[df_label['Patient ID'].isin(patient_ids)]
    df = df[df['Patient ID'].isin(patient_ids)]
    # Remove unncessary columns
    df_label.drop(['Patient ID','No Finding'], axis=1, inplace=True)  

    # List of images (full-path)
    img_locs =  df['Image Index'].map(lambda im: os.path.join(img_dir, im)).values
    # One-hot encoded labels (float32 for BCE loss)
    df_label['Image_path'] = img_locs
    return df_label


In [9]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER)

train:21563 valid:3080 test:6162


## Data Loading

### Creating the datasets

In [10]:
class XrayData(Dataset):
    def __init__(self, img_dir, lbl_file, patient_ids, transform=None):
        
        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.transform = transform
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
    
    def __getitem__(self, idx):
        im_file = self.img_locs[idx]
        im_rgb = Image.open(im_file)
        label = self.labels[idx]
        im_rgb = mx.nd.array(im_rgb)
        if self.transform is not None:
            im_rgb = self.transform(im_rgb)

        return im_rgb, mx.nd.array(label)
        
    def __len__(self):
        return len(self.img_locs)

In [11]:
def no_augmentation_dataset(img_dir, lbl_file, patient_ids, normalize):
    dataset = XrayData(img_dir, lbl_file, patient_ids,
                       transform=transforms.Compose([
                           transforms.Resize(WIDTH),
                           transforms.ToTensor(),  
                           transforms.Normalize(IMAGENET_RGB_MEAN, IMAGENET_RGB_SD)]))
    return dataset

In [12]:
# Dataset for training
train_dataset = XrayData(img_dir=IMAGE_FOLDER,
                         lbl_file=LABEL_FILE,
                         patient_ids=train_set,
                         transform=transforms.Compose([
                             transforms.RandomResizedCrop(size=WIDTH),
                             transforms.RandomFlipLeftRight(),
                             transforms.ToTensor(),
                             transforms.Normalize(IMAGENET_RGB_MEAN, IMAGENET_RGB_SD)]))

Loaded 87306 labels and 87306 images


In [13]:
valid_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, valid_set, transforms.Normalize(IMAGENET_RGB_MEAN, IMAGENET_RGB_SD))
test_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, test_set, transforms.Normalize(IMAGENET_RGB_MEAN, IMAGENET_RGB_SD))

Loaded 7616 labels and 7616 images
Loaded 17198 labels and 17198 images


In [14]:
# DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCHSIZE,
                          shuffle=True, num_workers=CPU_COUNT, last_batch='discard')
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCHSIZE,
                          shuffle=False, num_workers=CPU_COUNT, last_batch='discard')
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCHSIZE,
                         shuffle=False, num_workers=CPU_COUNT, last_batch='discard')

## Creating the network

### Loading the pretrained model

In [15]:
ctx = [mx.gpu(i) for i in range(GPU_COUNT)]   

In [16]:
net = mx.gluon.model_zoo.vision.densenet121(pretrained=True, ctx=ctx)
with net.name_scope():
    net.output = mx.gluon.nn.Dense(CLASSES)
net.output.initialize(ctx=ctx)
net.hybridize()

## Trainer

In [17]:
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': LR})

## Loss 

In [23]:
binary_cross_entropy = gluon.loss.SigmoidBinaryCrossEntropyLoss()

## Output

In [19]:
sig = gluon.nn.Activation('sigmoid')

## Evaluation loop

In [21]:
def evaluate_accuracy(data_iterator, net):
    acc = 0
    for i, (data, label) in enumerate(data_iterator):
        data_split = gluon.utils.split_and_load(data, ctx)
        label_split = gluon.utils.split_and_load(label, ctx)
        outputs = [(sig(net(X)),Y) for X, Y in zip(data_split, label_split)]
        for output, label in outputs:
            acc += float((label.asnumpy() == np.round(output.asnumpy())).sum()) / CLASSES / output.shape[0]
    data_split = gluon.utils.split_and_load(data, [mx.cpu()])
    label_split = gluon.utils.split_and_load(label, [mx.cpu()])
    return acc/i/len(ctx)

## Training loop

In [22]:
%%time
n_batch = 100
n_batches = len(train_loader)
for e in range(EPOCHS):
    tick = time()
    loss = 0
    for i, (data, label) in enumerate(train_loader):        
        data_split = gluon.utils.split_and_load(data, ctx)
        label_split = gluon.utils.split_and_load(label, ctx)  
        
        # Printing the loss here to allow data to be loaded asynchronously on the GPU
        if (i > 0):
            loss += sum(losses).mean().asscalar()
        if (i%n_batch == 0 and i > 0):
            print('Batch {0}: Sigmoid Binary Cross Entropy Loss: {1:.4f}'.format(i,loss/i))            
            
        with autograd.record():
            losses = [binary_cross_entropy(net(X), Y) for X, Y in zip(data_split, label_split)]
        for l in losses:
            l.backward()
        trainer.step(data.shape[0]) 
    test_accuracy = evaluate_accuracy(valid_loader, net)
    print('Epoch {0}, {1:.6f} test_accuracy after {2:.2f} seconds'.format(e, test_accuracy, time()-tick))

Batch 100: Sigmoid Binary Cross Entropy Loss: 0.8487
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.7617
Batch 300: Sigmoid Binary Cross Entropy Loss: 0.7285
Epoch 0, 0.985660 test_accuracy after 126.50 seconds
Batch 100: Sigmoid Binary Cross Entropy Loss: 0.6509
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.6428
Batch 300: Sigmoid Binary Cross Entropy Loss: 0.6409
Epoch 1, 0.985680 test_accuracy after 115.59 seconds
Batch 100: Sigmoid Binary Cross Entropy Loss: 0.6228
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.6241
Batch 300: Sigmoid Binary Cross Entropy Loss: 0.6262
Epoch 2, 0.985959 test_accuracy after 114.67 seconds
Batch 100: Sigmoid Binary Cross Entropy Loss: 0.6203
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.6182
Batch 300: Sigmoid Binary Cross Entropy Loss: 0.6200
Epoch 3, 0.985511 test_accuracy after 114.19 seconds
Batch 100: Sigmoid Binary Cross Entropy Loss: 0.6160
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.6134
Batch 300: Sigmoid Binary Cross Entropy Loss: 

## Evaluate

In [23]:
%%time
predictions = np.zeros((0, CLASSES))
labels = np.zeros((0, CLASSES))
for (data, label) in (test_loader):        
    data_split = gluon.utils.split_and_load(data, ctx)
    label_split = gluon.utils.split_and_load(label, ctx)  
    outputs = [sig(net(X)) for X in data_split]
    predictions = np.concatenate([predictions, np.concatenate([output.asnumpy() for output in outputs])])
    labels = np.concatenate([labels, np.concatenate([label.asnumpy() for label in label_split])])

CPU times: user 18 s, sys: 11.7 s, total: 29.7 s
Wall time: 14.8 s


In [24]:
print("Validation AUC: {0:.4f}".format(compute_roc_auc(labels, predictions, CLASSES)))

Full AUC [0.80908769611039455, 0.87312807723919572, 0.80886388318579316, 0.89052516742549404, 0.88370025678487119, 0.91963502792784457, 0.73497915084589172, 0.88040226735777372, 0.62529163818779587, 0.84906419830714797, 0.73238393223680975, 0.79835446517331077, 0.75521449476824731, 0.89192137073324274]
Validation AUC: 0.8180


## Synthetic Data (Pure Training)

In [25]:
fake_X = mx.nd.ones((tot_num, 3, 224, 224), dtype=np.float32)
fake_y = mx.nd.ones((tot_num, CLASSES), dtype=np.float32)

In [26]:
train_dataset_synth = ArrayDataset(fake_X, fake_y)
train_dataloader_synth = DataLoader(train_dataset_synth, BATCHSIZE, shuffle=False, num_workers=CPU_COUNT, last_batch='discard')

In [28]:
%%time
n_batch = 50
for e in range(EPOCHS):
    tick = time()
    loss = 0
    for i, (data, label) in enumerate(train_loader):        
        data_split = gluon.utils.split_and_load(data, ctx)
        label_split = gluon.utils.split_and_load(label, ctx)  
        
        # Printing the loss here to allow data to be loaded asynchronously on the GPU
        if (i > 0):
            loss += sum(losses).mean().asscalar()
        if (i%n_batch == 0 and i > 0):
            print('Batch {0}: Sigmoid Binary Cross Entropy Loss: {1:.4f}'.format(i,loss/i))            
            
        with autograd.record():
            losses = [binary_cross_entropy(net(X), Y) for X, Y in zip(data_split, label_split)]
        for l in losses:
            l.backward()
        trainer.step(data.shape[0]) 

    print('Epoch {0}, {1:.2f} seconds, loss {2:.4f}'.format(e, time()-tick, sum(losses).mean().asscalar()))


Batch 50: Sigmoid Binary Cross Entropy Loss: 0.6370
Batch 100: Sigmoid Binary Cross Entropy Loss: 0.6420
Batch 150: Sigmoid Binary Cross Entropy Loss: 0.6380
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.6385
Batch 250: Sigmoid Binary Cross Entropy Loss: 0.6373
Batch 300: Sigmoid Binary Cross Entropy Loss: 0.6377
Epoch 0, 106.00 seconds, loss 0.6637
Batch 50: Sigmoid Binary Cross Entropy Loss: 0.6239
Batch 100: Sigmoid Binary Cross Entropy Loss: 0.6236
Batch 150: Sigmoid Binary Cross Entropy Loss: 0.6264
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.6254
Batch 250: Sigmoid Binary Cross Entropy Loss: 0.6258
Batch 300: Sigmoid Binary Cross Entropy Loss: 0.6248
Epoch 1, 105.55 seconds, loss 0.6105
Batch 50: Sigmoid Binary Cross Entropy Loss: 0.6228
Batch 100: Sigmoid Binary Cross Entropy Loss: 0.6206
Batch 150: Sigmoid Binary Cross Entropy Loss: 0.6210
Batch 200: Sigmoid Binary Cross Entropy Loss: 0.6187
Batch 250: Sigmoid Binary Cross Entropy Loss: 0.6204
Batch 300: Sigmoid Binary Cr